### Conexión a Azure Data Lake y Selección del Archivo Más Reciente

In [2]:
from azure.storage.filedatalake import DataLakeServiceClient

# Cadena de conexión de Azure. La pongo como comentario porque no me deja subir a Git

# Crear el cliente del Data Lake
service_client = DataLakeServiceClient.from_connection_string(
    conn_str=connection_string, 
    api_version='2023-05-03'  # Versión soportada
)

# Conectar al sistema de archivos
file_system_client = service_client.get_file_system_client(file_system="datalake")

# Listar los archivos en la carpeta específica
paths = file_system_client.get_paths(path="silver/InventarioProductos/")

# Filtrar archivos con extensión .parquet y obtener el más reciente
latest_file = None
latest_time = None

for path in paths:
    if path.name.endswith('.parquet'):
        modified_time = path.last_modified
        
        if latest_time is None or modified_time > latest_time:
            latest_time = modified_time
            latest_file = path.name

# Mostrar el archivo más reciente
if latest_file:
    print(f"El archivo más reciente es: {latest_file}, modificado el: {latest_time}")
else:
    print("No se encontraron archivos .parquet en la carpeta especificada.")


El archivo más reciente es: silver/InventarioProductos/part-00000-6fd5cdc9-dcb5-462b-afa5-9f540a0d2a0c-c000.snappy.parquet, modificado el: 2024-09-18 10:13:19


### Descarga y Lectura del Archivo Parquet en un DataFrame de Pandas

In [4]:
import pandas as pd
from io import BytesIO

# Descargar el archivo
file_client = file_system_client.get_file_client(latest_file)
download = file_client.download_file()
downloaded_bytes = download.readall()

# Leer el archivo en un DataFrame de Pandas
df = pd.read_parquet(BytesIO(downloaded_bytes))

# Mostrar el DataFrame
print(df.head())

  ID_Producto Nombre_Producto Categoria_Producto Stock Precio_Costo  \
0           1            case            similar   412       120.36   
1          10           after              write    70       200.93   
2         100           again         difference   817       249.33   
3        1000           thank               onto   442        26.03   
4         101        thousand               wait   983        70.28   

  Precio_Venta ID_Proveedor                 Nombre_Proveedor  \
0       426.36          498  Miller, Castillo and Washington   
1        478.5          433                      Stevens Ltd   
2       407.38          404                        Olsen LLC   
3        60.95          497                     Clements LLC   
4       148.29          218                        Moran Ltd   

  Fecha_Ultimo_Abastecimiento  
0                  2023-10-22  
1                  2023-05-25  
2                  2023-10-10  
3                  2024-03-09  
4                  2024-03-2

### Configuración de MLflow y Librerías para el Modelado con Keras

In [6]:
# Importar librerías necesarias
import mlflow
import mlflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Configurar la URI local de seguimiento de MLflow
mlflow.set_tracking_uri("file:///./mlruns")  # Guardará el seguimiento en el directorio local
mlflow.set_experiment("Experimento_TFM")

print(f"MLFlow tracking URI: {mlflow.get_tracking_uri()}")

MLFlow tracking URI: file:///./mlruns


### Preprocesamiento de Datos: Conversión, Selección de Variables y Escalado

In [8]:
# Check columna Stock es de tipo numérico
df['Stock'] = pd.to_numeric(df['Stock'], errors='coerce')

# Seleccionar características y objetivo
X = df[['Stock', 'Precio_Costo', 'Precio_Venta']]
y = (df['Stock'] > 500).astype(int)

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

# Escalar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Definición, Entrenamiento y Registro del Modelo en MLflow

In [10]:
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Iniciar una nueva ejecución en MLflow
with mlflow.start_run() as run:
    # Entrenar
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))
    
    # Registrar el modelo en MLflow (localmente)
    mlflow.keras.log_model(model, "model")
    
    # Registrar las métricas en MLflow
    mlflow.log_metric("accuracy", history.history['accuracy'][-1])
    mlflow.log_metric("val_accuracy", history.history['val_accuracy'][-1])
    
    # Obtener el run_id de la ejecución actual
    run_id = run.info.run_id
    print(f"Modelo entrenado y registrado en MLflow con run_id: {run_id}")
    
    # Registrar el modelo en el registro de modelos (Model Registry)
    result = mlflow.register_model(
        f"runs:/{run_id}/model", 
        "Modelo-TFM"     
    )
    
    print(f"Modelo registrado en el registro de modelos: {result.name}, versión: {result.version}")

Epoch 1/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7415 - loss: 0.6280 - val_accuracy: 0.9433 - val_loss: 0.4988
Epoch 2/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9599 - loss: 0.4424 - val_accuracy: 0.9433 - val_loss: 0.3510
Epoch 3/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9802 - loss: 0.2811 - val_accuracy: 0.9767 - val_loss: 0.2359
Epoch 4/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9948 - loss: 0.1845 - val_accuracy: 0.9767 - val_loss: 0.1628
Epoch 5/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9880 - loss: 0.1332 - val_accuracy: 0.9800 - val_loss: 0.1222


2024/09/18 14:59:38 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/09/18 14:59:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo entrenado y registrado en MLflow con run_id: a2ef31e7856143179836de5b0d227b0c
Modelo registrado en el registro de modelos: Modelo-TFM, versión: 2


Registered model 'Modelo-TFM' already exists. Creating a new version of this model...
Created version '2' of model 'Modelo-TFM'.


### Carga el Modelo Registrado y Realizar Predicciones

In [12]:
# Cargar el modelo registrado usando el run_id obtenido
model_uri = f"runs:/{run_id}/model"
loaded_model = mlflow.keras.load_model(model_uri)

# Realizar predicciones con el modelo cargado
predictions = loaded_model.predict(X_test)

# Mostrar las predicciones
print(predictions)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
[[0.76305413]
 [0.24716915]
 [0.10889509]
 [0.9849169 ]
 [0.99731284]
 [0.67919195]
 [0.13315295]
 [0.25674388]
 [0.97420543]
 [0.98872083]
 [0.92494226]
 [0.9504586 ]
 [0.55104065]
 [0.90321445]
 [0.00635204]
 [0.9623132 ]
 [0.82846427]
 [0.02726847]
 [0.9952493 ]
 [0.02343801]
 [0.082097  ]
 [0.98957163]
 [0.9821023 ]
 [0.31765366]
 [0.03293265]
 [0.88328063]
 [0.04681096]
 [0.00981313]
 [0.98023176]
 [0.9939787 ]
 [0.47438058]
 [0.01470732]
 [0.00436798]
 [0.5890162 ]
 [0.897805  ]
 [0.60761356]
 [0.99209183]
 [0.9456571 ]
 [0.84599555]
 [0.03934225]
 [0.42864993]
 [0.9868711 ]
 [0.19927697]
 [0.9916674 ]
 [0.31723753]
 [0.03163141]
 [0.812595  ]
 [0.00630025]
 [0.03391916]
 [0.08563574]
 [0.16857165]
 [0.01060502]
 [0.0172473 ]
 [0.05107819]
 [0.99803585]
 [0.03081367]
 [0.6750443 ]
 [0.01864683]
 [0.9925496 ]
 [0.05460334]
 [0.00540701]
 [0.4090188 ]
 [0.51328605]
 [0.01460797]
 [0.07225708]
 [0.04241154]
 [0.04344097]
 [0.9940488 ]
 [0.8974

### Despliegue del Modelo con Flask para Predicciones en Tiempo Real

In [14]:
from flask import Flask, request, jsonify
import mlflow.keras
import pandas as pd
import threading

# Inicializar la aplicación Flask
app = Flask(__name__)

# Cargar el modelo desde MLflow
model_uri = f"runs:/{run_id}/model"  

model = mlflow.keras.load_model(model_uri)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json(force=True)
        input_data = pd.DataFrame(data)
        predictions = model.predict(input_data)
        return jsonify({'predictions': predictions.tolist()})
    except Exception as e:
        return jsonify({'error': str(e)})

# Función para ejecutar Flask en segundo plano
def run_flask():
    app.run(port=5003, debug=True, use_reloader=False)

# Ejecutar Flask en segundo plano dentro del notebook
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

print("Servidor Flask corriendo en http://localhost:5003")


Servidor Flask corriendo en http://localhost:5003
